In [2]:
import random
import math
from sys import maxsize
from itertools import permutations
import time


#-------------------------------------------------------------------> TSP with Greedy Algorithm
def greedyTSP(graph,s):
    # store all vertex apart from source vertex
    vertex = []
    for i in range(V):
        if i != s:
            vertex.append(i)

    # store minimum weight Hamiltonian Cycle
    min_path = maxsize
    next_permutation = permutations(vertex)
    for i in next_permutation:

        # store current Path weight(cost)
        current_pathweight = 0

        # compute current path weight
        k = s
        for j in i:
            current_pathweight += graph[k][j]
            k = j
        current_pathweight += graph[k][s]

        # update minimum
        min_path = min(min_path, current_pathweight)
    print(min_path)
#------------------------------------------------------------------X




#-------------------------------------------------------------------> TSP with Dynamic Programming
import numpy as np
from itertools import combinations
def dynamicTSP(G):
   n = len(G)
   C = [[np.inf for _ in range(n)] for __ in range(1 << n)]
   C[1][0] = 0 
   for size in range(1, n):
      for S in combinations(range(1, n), size):
        S = (0,) + S
        k = sum([1 << i for i in S])
        for i in S:
            if i == 0: continue
            for j in S:
                if j == i: continue
                cur_index = k ^ (1 << i)
                C[k][i] = min(C[k][i], C[cur_index][j]+ G[j][i])     
                                               #C[S−{i}][j]
   all_index = (1 << n) - 1
   print(min([(C[all_index][i] + G[0][i], i) for i in range(n)])[0])
#------------------------------------------------------------------X



#-------------------------------------------------------------------> TSP with Branch and Bound
# Function to copy temporary solution
# to the final solution
def copyToFinal(curr_path):
    final_path[:N + 1] = curr_path[:]
    final_path[N] = curr_path[0]


# Function to find the minimum edge cost
# having an end at the vertex i
def firstMin(adj, i):
    min = maxsize
    for k in range(N):
        if adj[i][k] < min and i != k:
            min = adj[i][k]

    return min


# function to find the second minimum edge
# cost having an end at the vertex i
def secondMin(adj, i):
    first, second = maxsize, maxsize
    for j in range(N):
        if i == j:
            continue
        if adj[i][j] <= first:
            second = first
            first = adj[i][j]

        elif (adj[i][j] <= second and
              adj[i][j] != first):
            second = adj[i][j]

    return second


# function that takes as arguments:
# curr_bound -> lower bound of the root node
# curr_weight-> stores the weight of the path so far
# level-> current level while moving
# in the search space tree
# curr_path[] -> where the solution is being stored
# which would later be copied to final_path[]
def TSPRec(adj, curr_bound, curr_weight,level, curr_path, visited):
    global final_res

    # base case is when we have reached level N
    # which means we have covered all the nodes once
    if level == N:

        # check if there is an edge from
        # last vertex in path back to the first vertex
        if adj[curr_path[level - 1]][curr_path[0]] != 0:

            # curr_res has the total weight
            # of the solution we got
            curr_res = curr_weight + adj[curr_path[level - 1]] \
                [curr_path[0]]
            if curr_res < final_res:
                copyToFinal(curr_path)
                final_res = curr_res
        return

    # for any other level iterate for all vertices
    # to build the search space tree recursively
    for i in range(N):

        # Consider next vertex if it is not same
        # (diagonal entry in adjacency matrix and
        #  not visited already)
        if (adj[curr_path[level - 1]][i] != 0 and
                visited[i] == False):
            temp = curr_bound
            curr_weight += adj[curr_path[level - 1]][i]

            # different computation of curr_bound
            # for level 2 from the other levels
            if level == 1:
                curr_bound -= ((firstMin(adj, curr_path[level - 1]) +
                                firstMin(adj, i)) / 2)
            else:
                curr_bound -= ((secondMin(adj, curr_path[level - 1]) +
                                firstMin(adj, i)) / 2)

            # curr_bound + curr_weight is the actual lower bound
            # for the node that we have arrived on.
            # If current lower bound < final_res,
            # we need to explore the node further
            if curr_bound + curr_weight < final_res:
                curr_path[level] = i
                visited[i] = True

                # call TSPRec for the next level
                TSPRec(adj, curr_bound, curr_weight,
                       level + 1, curr_path, visited)

            # Else we have to prune the node by resetting
            # all changes to curr_weight and curr_bound
            curr_weight -= adj[curr_path[level - 1]][i]
            curr_bound = temp

            # Also reset the visited array
            visited = [False] * len(visited)
            for j in range(level):
                if curr_path[j] != -1:
                    visited[curr_path[j]] = True


# This function sets up final_path
def branchAndBoundTSP(adj):
    # Calculate initial lower bound for the root node
    # using the formula 1/2 * (sum of first min +
    # second min) for all edges. Also initialize the
    # curr_path and visited array
    curr_bound = 0
    curr_path = [-1] * (N + 1)
    visited = [False] * N

    # Compute initial bound
    for i in range(N):
        curr_bound += (firstMin(adj, i) +
                       secondMin(adj, i))

    # Rounding off the lower bound to an integer
    curr_bound = math.ceil(curr_bound / 2)

    # We start at vertex 1 so the first vertex
    # in curr_path[] is 0
    visited[0] = True
    curr_path[0] = 0

    # Call to TSPRec for curr_weight
    # equal to 0 and level 1
    TSPRec(adj, curr_bound, 0, 1, curr_path, visited)
    print(final_res)

#------------------------------------------------------------------X


#------------------------------------------------------------------> TSP with Backtracking
# Function to find the minimum weight
# Hamiltonian Cycle
def backtrackTSP(graph, v, currPos, n, count, cost):
    # If last node is reached and it has
    # a link to the starting node i.e
    # the source then keep the minimum
    # value out of the total cost of
    # traversal and "ans"
    # Finally return to check for
    # more possible values
    if (count == n and graph[currPos][0]):
        answer.append(cost + graph[currPos][0])
        return

    # BACKTRACKING STEP
    # Loop to traverse the adjacency list
    # of currPos node and increasing the count
    # by 1 and cost by graph[currPos][i] value
    for i in range(n):
        if (v[i] == False and graph[currPos][i]):
            # Mark as visited
            v[i] = True
            backtrackTSP(graph, v, i, n, count + 1,
                cost + graph[currPos][i])

            # Mark ith node as unvisited
            v[i] = False


#------------------------------------------------------------------X


dfDictionary={
    "N":[],
    "GreedyTSPtime":[],
    "DynamicTSPtime":[],
    "BranchAndBoundTSPtime":[],
    "BactrackTSPtime":[]

}

#Node is i and i+1 is number of nodes in map

for i in range(2, 12):
    map = dict({})
    matrix = []
    for j in range(0, i+1):
        if j not in map:
            map[j] = []
        for k in range(0,i+1):
            if k  < j:
                map[j].append(map[k][j])
            elif k == j:
                map[j].append(0)
            else:
                map[j].append(random.randint(1, i))
        matrix.append(map[j])


    print("----------------------------------------------------------> Iteration for",i+1,"Nodes")
    dfDictionary["N"].append(i+1)
    
    
    
#-------------------------------> Call Greedy TSP
    s = 0
    V=i+1
    t1=time.perf_counter_ns()
    print("Greedy TSP Minimum path: ",end="")
    greedyTSP(matrix.copy(),s)
    t2= time.perf_counter_ns()
    print("Greedy TSP Time: ",t2-t1)
    dfDictionary["GreedyTSPtime"].append(t2-t1)
#-------------------------------X
    
    

#-------------------------------> Call Dynamic Programming TSP
    s = 0
    V=i+1
    t1=time.perf_counter_ns()
    print("Dynamic TSP Minimum path: ",end="")
    dynamicTSP(matrix.copy())
    t2= time.perf_counter_ns()
    print("Dynamic TSP Time: ",t2-t1)
    dfDictionary["DynamicTSPtime"].append(t2-t1)
#-------------------------------X


# -------------------------------> Call Branch and Bound TSP
    N = i+1
    # final_path[] stores the final solution
    # i.e. the // path of the salesman.
    final_path = [None] * (N + 1)

    # visited[] keeps track of the already
    # visited nodes in a particular path
    visited = [False] * N

    # Stores the final minimum weight
    # of shortest tour.
    final_res = maxsize
    print("Branch and Bound TSP Minimum path:", end="")
    t1 = time.perf_counter_ns()
    branchAndBoundTSP(matrix.copy())
    t2 = time.perf_counter_ns()
    print("Branch and Bound TSP Time:",t2-t1)
    dfDictionary["BranchAndBoundTSPtime"].append(t2-t1)
#-------------------------------X


# -------------------------------> Call  Backtrak TSP
    V = i+1
    n=i+1
    answer = []
    v = [False for i in range(n)]
    v[0] = True
    t1 = time.perf_counter_ns()
    backtrackTSP(matrix.copy(), v, 0, n, 1, 0)
    t2 = time.perf_counter_ns()
    print("Backtrack TSP Minimum path:",min(answer))
    print("Backtrack TSP Time:",t2-t1)
    dfDictionary["BactrackTSPtime"].append(t2-t1)
# -------------------------------X

    print("----------------------------------------------------------X")

----------------------------------------------------------> Iteration for 3 Nodes
Greedy TSP Minimum path: 4
Greedy TSP Time:  41000
Dynamic TSP Minimum path: 4
Dynamic TSP Time:  50900
Branch and Bound TSP Minimum path:4
Branch and Bound TSP Time: 47900
Backtrack TSP Minimum path: 4
Backtrack TSP Time: 7400
----------------------------------------------------------X
----------------------------------------------------------> Iteration for 4 Nodes
Greedy TSP Minimum path: 6
Greedy TSP Time:  22900
Dynamic TSP Minimum path: 6
Dynamic TSP Time:  96000
Branch and Bound TSP Minimum path:6
Branch and Bound TSP Time: 956700
Backtrack TSP Minimum path: 6
Backtrack TSP Time: 8900
----------------------------------------------------------X
----------------------------------------------------------> Iteration for 5 Nodes
Greedy TSP Minimum path: 11
Greedy TSP Time:  59100
Dynamic TSP Minimum path: 11
Dynamic TSP Time:  105000
Branch and Bound TSP Minimum path:11
Branch and Bound TSP Time: 103700

In [3]:
import numpy as np
import pandas as pd
df = pd.DataFrame(dfDictionary)
df.to_csv("TSPtimeData.csv")

In [4]:
df

,N,GreedyTSPtime,DynamicTSPtime,BranchAndBoundTSPtime,BactrackTSPtime
0,3,41000,50900,47900,7400
1,4,22900,96000,956700,8900
2,5,59100,105000,103700,41000
3,6,87400,177700,398800,151100
4,7,452600,381000,230100,911200
5,8,2887700,994600,920200,6374900
6,9,29645500,4080300,3690200,49866500
7,10,239350500,5464800,32544200,495057700
8,11,2848791000,12073900,22417700,5387322500
9,12,34543578000,50251500,452681700,64829084900
